# 1. A function for reading files

In [2]:
import os

def readtxt(path, encoding):
    with open(path, 'r', encoding = encoding) as f:
        lines = f.readlines()
    return lines

def fileWalker(path):
    fileArray = []
    for root, dirs, files in os.walk(path):
        for fn in files:
            # eachpath = str(root + '/' + fn) #Linux
            eachpath = os.path.join(root, fn)  # Automatically adapts to separators of Windows/Linux/Mac
            fileArray.append(eachpath)
    return fileArray

# 2. Use regular expressions to split text

In [3]:
def email_parser(email_path):
    punctuations = """,.<>()*&^%$#@!'";~`[]{}|、\\/~+_-=?"""
    content_list = readtxt(email_path, 'utf8')
    content = (' '.join(content_list)).replace('\r\n', ' ').replace('\t', ' ')
    clean_word = []
    # for punctuation in punctuations:
    #     content = (' '.join(content.split(punctuation))).replace('  ', ' ')
    #     clean_word = [word.lower()
    #                   for word in content.split(' ') if len(word) > 2]
    # Modify to avoid repeated generation
    for punctuation in punctuations:
        content = ' '.join(content.split(punctuation))
    while '  ' in content:
        content = content.replace('  ', ' ')
    clean_word = [word.lower() for word in content.split(' ') if len(word) > 2]
    return clean_word

# 3. Create a dictionary to count word frequencies

In [9]:
def count_word_prob(email_list, union_set):
    word_prob = {}
    for word in union_set:
        counter = 0
        for email in email_list:
            if word in email:
                counter += 1
            else:
                continue
        prob = 0.0
        if counter != 0:
            prob = counter / len(email_list)
        else:
            prob = 0.01
        word_prob[word] = prob
    return word_prob

def get_word(email_file):
    word_list = []
    word_set = []
    email_paths = fileWalker(email_file)
    for email_path in email_paths:
        clean_word = email_parser(email_path)
        word_list.append(clean_word)
        word_set.extend(clean_word)
    return word_list, set(word_set)

# 4. Implementation of Naive Bayes Algorithm for Spam Filtering

In [11]:
def myfilter(ham_word_pro, spam_word_pro, test_file):
    test_paths = fileWalker(test_file)
    for test_path in test_paths:
        email_spam_prob = 0.0 # Initialization
        # prior probability:
        spam_prob = 0.5 # P(spam)
        ham_prob = 0.5 # P(ham)

        file_name = test_path.split('\\')[-1]
        prob_dict = {}
        words = set(email_parser(test_path)) # Store the probability that emails containing each word are spam, corresponding to each word.
        for word in words:
            Psw = 0.0 # Initialize P(spam|word) to 0.0
            # Case 1: If the word is not in the spam word probability dictionary (has not appeared in the training set), assign a default value of 0.4
            if word not in spam_word_pro:
                Psw = 0.4
            # Case 2: The word has appeared in the training set, and the Bayesian formula is used to calculate P(spam | word)
            else:
                # Extract the conditional probabilities calculated from the training set
                Pws = spam_word_pro[word] # P(word|spam)
                Pwh = ham_word_pro[word] # P(word|ham)

                # Core formula: Bayes' theorem for calculating P(spam|word)
                # P(spam|word) = [P(word|spam) * P(spam)] / [P(word|normal email) * P(normal email) + P(word|spam) * P(spam)]
                Psw = spam_prob * (Pws / ((Pwh * ham_prob + Pws * spam_prob)))
            prob_dict[word] = Psw
        # Combine the probabilities of all words to calculate the final probability that the email is a spam.
        numerator = 1
        denominator_h = 1
        # Iterate through the probabilities of all words and calculate the product
        for k, v in prob_dict.items():
            numerator *= v
            denominator_h *= (1 - v)
        # Final probability: P(spam | all words) = numerator / (numerator + denominator normal term)
        email_spam_prob = round(numerator / (numerator + denominator_h), 4)

        if email_spam_prob > 0.5:
            print(file_name, 'spam', email_spam_prob)
        else:
            print(file_name, 'ham', email_spam_prob)
        
        print(prob_dict)
        print('***********************************************************')

# 5. Main function call

In [12]:
def main():
    ham_file = 'data/email/ham'
    spam_file = 'data/email/spam'
    test_file = 'data/email/test'
    ham_list, ham_set = get_word(ham_file)
    spam_list, spam_set = get_word(spam_file)
    union_set = ham_set | spam_set
    ham_word_pro = count_word_prob(ham_list, union_set)
    spam_word_pro = count_word_prob(spam_list, union_set)
    myfilter(ham_word_pro, spam_word_pro, test_file)

if __name__ == '__main__':
    main()

1.txt ham 0.0
{'keep': 0.19999999999999998, 'town': 0.19999999999999998, 'and': 0.5, 'while': 0.19999999999999998, 'out': 0.11111111111111112, 'going': 0.058823529411764705, 'jose': 0.19999999999999998, 'peter': 0.034482758620689655, 'eugene': 0.19999999999999998, 'to\n': 0.19999999999999998, 'stuff': 0.11111111111111112, 'interesting': 0.11111111111111112, 'things\n': 0.19999999999999998, 'with': 0.125, 'meet': 0.19999999999999998, 'let': 0.33333333333333337, 'you': 0.4230769230769231, 'want': 0.19999999999999998, 'some': 0.07692307692307693, 'once': 0.19999999999999998, 'know\n': 0.19999999999999998}
***********************************************************
10.txt spam 1.0
{'prices': 0.6666666666666667, 'drugs': 0.888888888888889, 'superb': 0.888888888888889, 'cards': 0.888888888888889, 'and': 0.5, 'required\n': 0.923076923076923, 'online': 0.8750000000000001, 'canadian': 0.923076923076923, 'fda': 0.888888888888889, 'noprescription': 0.923076923076923, 'ordercializviagra': 0.888888